# ETL file to convert the geojson file into network files

In [1]:
try:
    import json 
    import csv
    import os
    from math import sin, cos, sqrt, atan2, radians

    import networkx as nx

    from folium import *

    import pandas as pd

    
except:
    %pip install folium
    %pip install pandas
    %pip install networkx

In [5]:
class ParisTransportation:
    @staticmethod
    def clear():
        if os.name == 'posix':
            os.system('clear')
        else:
            os.system('cls')

    @staticmethod
    def clean_data():
        nodefile = "vertex/vertex.csv"
        edgefile = "edge/edge.csv"
        jsonfile = "geojson/multigraph.geojson"
        graphmlfile = "graphml/multigraph.graphml"
        gmlfile = "gml/multigraph.gml"

        if not os.path.exists("edge"):
            os.makedirs("edge")
        if not os.path.exists("vertex"):
            os.makedirs("vertex")
        if not os.path.exists("gml"):
            os.makedirs("gml")
        
        with open(jsonfile, 'r') as jsfile:
            with open(nodefile, 'w+') as nodefiled:
                with open(edgefile, 'w+') as edgefiled:
                    node = csv.writer(nodefiled)
                    edge = csv.writer(edgefiled)
                    # Header
                    node.writerow(["# NodeID", "Lat", " Lon", "Layer"])
                    edge.writerow(["# EdgeID", "Source NodeID", "Target NodeID", "Direction", "Layer"])
                    for line in jsfile:
                        jsentry = json.loads(line)
                        if jsentry['properties']['type'] == "node":
                            node.writerow([
                                    jsentry['_id']['$oid'], 
                                    jsentry['geometry']['coordinates'][0], 
                                    jsentry['geometry']['coordinates'][1], 
                                    jsentry['properties']['layer']])
                        if jsentry['properties']['type'] == "edge":
                            if jsentry['properties']['name'].startswith("54"):
                                jsentry['properties']['name'] = "None"
                            if 'direction' in jsentry['properties']:
                                if jsentry['properties']['direction'] == "Double sens":
                                    direction = "TwoWay"
                                elif (jsentry['properties']['direction'] == "Sens inverse" 
                                    or jsentry['properties']['direction'] == "Sens unique"):
                                    direction = "OneWay"
                                    
                                edge.writerow([
                                    jsentry['properties']['mongo_org_id'],
                                    jsentry['properties']['mongo_dest_id'],
                                    jsentry['_id']['$oid'], 
                                    direction,
                                    jsentry['properties']['layer'], 
                                    jsentry['properties']['name']])
                            else:
                                edge.writerow([
                                    jsentry['properties']['mongo_org_id'],
                                    jsentry['properties']['mongo_dest_id'],
                                    jsentry['_id']['$oid'],
                                    "TwoWay",
                                    jsentry['properties']['layer'], 
                                    jsentry['properties']['name']])

        G = nx.DiGraph()

        with open(nodefile, 'r') as node:
            reader = csv.reader(node)
            next(reader)
            for row in reader:
                if len(row) != 0:
                    lat = row[1]
                    lon = row[2]
                    ntype = row[3]
                    G.add_node(row[0], lat=lat, lon=lon, type=ntype)

        with open(edgefile, 'r') as node:
            reader = csv.reader(node)
            next(reader)
            for row in reader:
                if len(row) != 0:
                    G.add_edge(row[1], row[2], type=row[4], name=row[5])
                    if row[3] == 'TwoWay':
                        G.add_edge(row[2], row[1], type=row[4], name=row[5])


        nx.write_graphml(G, graphmlfile)
        nx.write_graphml(G, gmlfile)
        return(G)

    def geoJSON(self):
        df = pd.read_json('geojson/multigraph.geojson', lines=True)
        id = pd.json_normalize(df["_id"])
        id = id.rename(columns={"$oid": "ID"})

        geom = pd.json_normalize(df["geometry"])
        geom = geom.rename(columns={"type": "LP"})

        prop = pd.json_normalize(df["properties"])

        new_df = pd.concat([id, geom, prop], axis=1)
        return(new_df)
    
    class MapMaker:
        def __init__(self, df):
            self.df = df
            self.only_point = df.loc[df['LP'] == "Point"]
            self.points = self.only_point.coordinates.tolist()
            self.center = (48.85654066902656, 2.349154275836)
            self.canvas = True,
            self.bounds = True,
            self.filename = "france.html"
            self.layerColor = {
                "road": "green",
                "train": "blue",
                "metro": "gray",
                "tram": "purple"
            }

        @staticmethod
        def calculate_distance(lon1, lat1, lon2, lat2, R=6373.0):
            lat1 = radians(lat1)
            lat2 = radians(lat2)
            lon1 = radians(lon1)
            lon2 = radians(lon2)

            dlon = lon2 - lon1
            dlat = lat2 - lat1

            a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
            c = 2 * atan2(sqrt(a), sqrt(1 - a))

            distance = R * c
            return distance
        
        def create_map(self):
            if not os.path.exists(self.filename):
                m = Map(location=[self.center[0], self.center[1]], 
                    tiles="CartoDB positron", 
                    min_zoom=7, 
                    zoom_start=9,
                    zoom_control=True, 
                    min_lat=42, 
                    max_lat=54, 
                    min_lon=-10, 
                    max_lon=14, 
                    max_bounds=self.bounds,
                    prefer_canvas=self.canvas)

                minimap = plugins.MiniMap()
                m.add_child(minimap)

                layers = self.only_point.layer.unique().tolist()

                fg = FeatureGroup(control=False, show=False)
                m.add_child(fg)

                f1 = plugins.FeatureGroupSubGroup(fg, layers[0].capitalize())
                m.add_child(f1)

                f2 = plugins.FeatureGroupSubGroup(fg, layers[1].capitalize())
                m.add_child(f2)

                f3 = plugins.FeatureGroupSubGroup(fg, layers[2].capitalize())
                m.add_child(f3)

                f4 = plugins.FeatureGroupSubGroup(fg, layers[3].capitalize())
                m.add_child(f4)

                i = 0
                layerTypes = self.only_point.layer.tolist()

                for lon, lat in self.points:

                    distance = self.calculate_distance(lon, lat, self.center[1], self.center[0])
                    radiusDistance = distance

                    if int(distance) > 0:
                        distance = int(distance)
                        distance = f"{distance} km"
                    else:
                        distance = distance * 1000
                        distance = int(distance)
                        distance = f"{distance} mt"

                    circle = Circle(
                        location=(lat, lon),
                        tooltip=f"<strong>Type:</strong> {layerTypes[i]}<br><strong>Center distance:</strong> {distance}",
                        radius=sqrt(radiusDistance * 1000),
                        popup=f"<strong>Type:</strong> {layerTypes[i]}<br><strong>Lat:</strong> {lat}<br><strong>Long:</strong> {lon}<br><strong>Center distance:</strong> {distance}",
                        color=self.layerColor.get(layerTypes[i]),
                        fill=False,
                        fill_color=self.layerColor.get(layerTypes[i])
                    )
                    
                    if layerTypes[i] == layers[0]:
                        f1.add_child(circle)
                    elif layerTypes[i] == layers[1]:
                        f2.add_child(circle)
                    elif layerTypes[i] == layers[2]:
                        f3.add_child(circle)
                    elif layerTypes[i] == layers[3]:
                        f4.add_child(circle)

                    i +=1

                paris = Circle(
                    location=(self.center[0], self.center[1]),
                    tooltip="Center",
                    popup="",
                    radius=80,
                    color="crimson",
                    fill=True,
                    fill_color="crimson"
                )
                paris.add_to(fg)

                plugins.Fullscreen(
                        position="topright",
                        title="Fullscreen",
                        title_cancel="Exit fullscreen",
                        force_separate_button=True,
                    ).add_to(m)

                LayerControl().add_to(m)
                m.save(self.filename)

In [6]:
#only_point = new_df.loc[new_df['LP'] == "Point"]
#only_point.loc[only_point['layer']=="tram"]
#only_point.layer.unique().tolist()

['road', 'train', 'metro', 'tram']

In [6]:
G = ParisTransportation().clean_data()
df = ParisTransportation().geoJSON()
ParisTransportation().MapMaker(df).create_map()

In [11]:
len(G.nodes())
len(G.edges())

51931